In [1]:
import numpy as np
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [2]:
from keras import backend as K
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Conv2D, Flatten, MaxPooling2D, Reshape, Activation, Embedding, TimeDistributed
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
import pandas as pd

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

/home/zhouz2/.conda/envs/rbenv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
df = pd.read_csv('reuters.csv', delimiter='\t', names=['id', 'timestamp', 'title', 'url', 'first_line'])

In [4]:
df = df.set_index('id')

In [5]:
batch_size = 64  # Batch size for training.
epochs = 10000  # Number of epochs to train for.
latent_dim = 1024  # Latent dimensionality of the encoding space (number of nodes per LSTM layer).
num_samples = min(10, len(df))  # Number of samples to train on.
num_lstm = 4 # Number of LSTM layers in the encoder and decoder.

In [6]:
input_texts = df['first_line'][:num_samples]
output_texts = df['title'][:num_samples]

In [7]:
input_texts = input_texts.apply(lambda i: i.strip().lower() + ' \n')
output_texts = output_texts.apply(lambda o: '\t '+ o.strip().lower() + ' \n')

In [8]:
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~', oov_token='<unk>')
tokenizer.fit_on_texts(input_texts)
tokenizer.fit_on_texts(output_texts)

In [25]:
input_sequences = tokenizer.texts_to_sequences(input_texts)
output_sequences = tokenizer.texts_to_sequences(output_texts)
word_id_dict = tokenizer.word_index
id_word_dict = dict()
for k in word_id_dict:
    id_word_dict[word_id_dict[k]] = k;

In [10]:
num_samples = len(input_texts)
max_encoder_seq_length = max([len(seq) for seq in input_sequences])
max_decoder_seq_length = max([len(seq) for seq in output_sequences])
num_dict_size = len(tokenizer.word_index)
# num_input_tokens = len(input_dict)
# num_output_tokens = len(output_dict)

In [11]:
print('Number of samples:', num_samples)
print('Max sequence length for inputs:', max_encoder_seq_length)
# print('Num of tokens for inputs:', num_input_tokens)
print('Max sequence length for outputs:', max_decoder_seq_length)
# print('Num of tokens for outputs:', num_output_tokens)
print('Number of words in the dictionary (including OOV token):', num_dict_size)

Number of samples: 10
Max sequence length for inputs: 42
Max sequence length for outputs: 14
Number of words in the dictionary (including OOV token): 258


In [12]:
# encoder_input_data = np.zeros((num_samples, max_encoder_seq_length, num_input_tokens),dtype='float32')
# reversed_encoder_input_data = np.zeros((num_samples, max_encoder_seq_length, num_input_tokens),dtype='float32')
# decoder_input_data = np.zeros((num_samples, max_decoder_seq_length, num_output_tokens),dtype='float32')
# decoder_target_data = np.zeros((num_samples, max_decoder_seq_length, num_output_tokens),dtype='float32')

In [13]:
# for i in range(num_samples):
#     input_text = input_tokenizer.texts_to_sequences(input_texts[i])
#     input_text = input_dict
#     encoder_input_data[i, :, :] = input_tokenizer.texts_to_sequences(input_texts[i])
#     reversed_encoder_input_data[i, :, :] = encoder_input_data[i, ::-1, :]
#     decoder_input_data[i, :, :] = output_tokenizer.texts_to_sequences(output_texts[i])
#     decoder_target_data[i, :, :] = np.row(decoder_input_data[i, :, :], 1)
#     decoder_target_data[i, 0, :] = 0;

In [67]:
## GNMT variant 1: Dense + Activation to get the whole sentence, instead of softmax probability of each word
# Define an input sequence and process it.
encoder_inputs = Input(shape=(max_encoder_seq_length,))
# print(encoder.shape)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Attention module

# Decoder module
decoder_inputs = Input(shape=(max_decoder_seq_length,))
decoder_embedding = Embedding(num_dict_size, latent_dim)
decoder = decoder_embedding(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder, _, _ = decoder_lstm(decoder, initial_state=encoder_states)
decoder = Flatten()(decoder)
decoder_dense = Dense(14, activation='relu')
decoder_outputs = decoder_dense(decoder)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)# Compile & run training
# model.compile(optimizer='adam', loss='mse')

# model.summary()
# SVG(model_to_dot(model).create(prog='dot', format='svg'))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 42)           0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 14)           0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 42, 256)      66048       input_17[0][0]                   
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, 14, 256)      66048       input_18[0][0]                   
__________________________________________________________________________________________________
lstm_13 (L

In [15]:
encoder_input_data = np.zeros((num_samples, max_encoder_seq_length), dtype='int32')
decoder_input_data = np.zeros((num_samples, max_decoder_seq_length), dtype='int32')
decoder_target_data = np.zeros((num_samples, max_decoder_seq_length), dtype='int32')

In [16]:
for k in range(num_samples):
    encoder_input_data[k][:len(input_sequences[k])] = input_sequences[k][:]
    decoder_input_data[k][:len(output_sequences[k])] = output_sequences[k][:]
    decoder_target_data[k][:len(output_sequences[k])-1] = output_sequences[k][1:]

In [17]:
# encoder_input_data = encoder_input_data.reshape((*encoder_input_data.shape, 1))
# decoder_input_data = decoder_input_data.reshape((*decoder_input_data.shape, 1))
# decoder_target_data = decoder_target_data.reshape((*decoder_target_data.shape, 1))

In [18]:
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs#,
          #validation_split=0.2
         )

Epoch 1/10000
10/10 [==============================] - 3s 276ms/step - loss: 11758.9434
Epoch 2/10000
10/10 [==============================] - 0s 10ms/step - loss: 11751.5098
Epoch 3/10000
10/10 [==============================] - 0s 12ms/step - loss: 11741.2012
Epoch 4/10000
10/10 [==============================] - 0s 11ms/step - loss: 11725.6650
Epoch 5/10000
10/10 [==============================] - 0s 12ms/step - loss: 11699.4033
Epoch 6/10000
10/10 [==============================] - 0s 10ms/step - loss: 11649.3467
Epoch 7/10000
10/10 [==============================] - 0s 8ms/step - loss: 11534.6982
Epoch 8/10000
10/10 [==============================] - 0s 10ms/step - loss: 11210.8447
Epoch 9/10000
10/10 [==============================] - 0s 10ms/step - loss: 10634.6973
Epoch 10/10000
10/10 [==============================] - 0s 10ms/step - loss: 10382.6045
Epoch 11/10000
10/10 [==============================] - 0s 11ms/step - loss: 10262.4551
Epoch 12/10000
10/10 [===================

Epoch 96/10000
10/10 [==============================] - 0s 8ms/step - loss: 8064.6182
Epoch 97/10000
10/10 [==============================] - 0s 11ms/step - loss: 8055.5205
Epoch 98/10000
10/10 [==============================] - 0s 9ms/step - loss: 8046.5547
Epoch 99/10000
10/10 [==============================] - 0s 14ms/step - loss: 8037.7217
Epoch 100/10000
10/10 [==============================] - 0s 11ms/step - loss: 8029.0181
Epoch 101/10000
10/10 [==============================] - 0s 10ms/step - loss: 8020.4404
Epoch 102/10000
10/10 [==============================] - 0s 10ms/step - loss: 8011.9907
Epoch 103/10000
10/10 [==============================] - 0s 9ms/step - loss: 8003.6650
Epoch 104/10000
10/10 [==============================] - 0s 10ms/step - loss: 7995.4634
Epoch 105/10000
10/10 [==============================] - 0s 11ms/step - loss: 7987.3818
Epoch 106/10000
10/10 [==============================] - 0s 12ms/step - loss: 7979.4229
Epoch 107/10000
10/10 [================

Epoch 190/10000
10/10 [==============================] - 0s 11ms/step - loss: 7617.9468
Epoch 191/10000
10/10 [==============================] - 0s 12ms/step - loss: 7616.1221
Epoch 192/10000
10/10 [==============================] - 0s 10ms/step - loss: 7614.3345
Epoch 193/10000
10/10 [==============================] - 0s 13ms/step - loss: 7612.5820
Epoch 194/10000
10/10 [==============================] - 0s 14ms/step - loss: 7610.8633
Epoch 195/10000
10/10 [==============================] - 0s 14ms/step - loss: 7609.1807
Epoch 196/10000
10/10 [==============================] - 0s 12ms/step - loss: 7607.5312
Epoch 197/10000
10/10 [==============================] - 0s 12ms/step - loss: 7605.9141
Epoch 198/10000
10/10 [==============================] - 0s 12ms/step - loss: 7604.3311
Epoch 199/10000
10/10 [==============================] - 0s 10ms/step - loss: 7602.7798
Epoch 200/10000
10/10 [==============================] - 0s 14ms/step - loss: 7601.2603
Epoch 201/10000
10/10 [=========

10/10 [==============================] - 0s 9ms/step - loss: 7544.2954
Epoch 284/10000
10/10 [==============================] - 0s 10ms/step - loss: 7544.0845
Epoch 285/10000
10/10 [==============================] - 0s 10ms/step - loss: 7543.8789
Epoch 286/10000
10/10 [==============================] - 0s 11ms/step - loss: 7543.6797
Epoch 287/10000
10/10 [==============================] - 0s 11ms/step - loss: 7543.4844
Epoch 288/10000
10/10 [==============================] - 0s 9ms/step - loss: 7543.2939
Epoch 289/10000
10/10 [==============================] - 0s 10ms/step - loss: 7543.1094
Epoch 290/10000
10/10 [==============================] - 0s 10ms/step - loss: 7542.9297
Epoch 291/10000
10/10 [==============================] - 0s 10ms/step - loss: 7542.7539
Epoch 292/10000
10/10 [==============================] - 0s 10ms/step - loss: 7542.5845
Epoch 293/10000
10/10 [==============================] - 0s 10ms/step - loss: 7542.4180
Epoch 294/10000
10/10 [===========================

Epoch 377/10000
10/10 [==============================] - 0s 11ms/step - loss: 7537.0718
Epoch 378/10000
10/10 [==============================] - 0s 11ms/step - loss: 7537.0557
Epoch 379/10000
10/10 [==============================] - 0s 9ms/step - loss: 7537.0405
Epoch 380/10000
10/10 [==============================] - 0s 11ms/step - loss: 7537.0249
Epoch 381/10000
10/10 [==============================] - 0s 12ms/step - loss: 7537.0107
Epoch 382/10000
10/10 [==============================] - 0s 10ms/step - loss: 7536.9961
Epoch 383/10000
10/10 [==============================] - 0s 14ms/step - loss: 7536.9814
Epoch 384/10000
10/10 [==============================] - 0s 14ms/step - loss: 7536.9673
Epoch 385/10000
10/10 [==============================] - 0s 14ms/step - loss: 7536.9546
Epoch 386/10000
10/10 [==============================] - 0s 12ms/step - loss: 7536.9404
Epoch 387/10000
10/10 [==============================] - 0s 12ms/step - loss: 7536.9282
Epoch 388/10000
10/10 [==========

...

Epoch 9553/10000
10/10 [==============================] - 0s 8ms/step - loss: 0.1730
Epoch 9554/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1730
Epoch 9555/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1729
Epoch 9556/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1729
Epoch 9557/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1729
Epoch 9558/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1729
Epoch 9559/10000
10/10 [==============================] - 0s 9ms/step - loss: 0.1728
Epoch 9560/10000
10/10 [==============================] - 0s 12ms/step - loss: 0.1728
Epoch 9561/10000
10/10 [==============================] - 0s 12ms/step - loss: 0.1728
Epoch 9562/10000
10/10 [==============================] - 0s 9ms/step - loss: 0.1728
Epoch 9563/10000
10/10 [==============================] - 0s 8ms/step - loss: 0.1727
Epoch 9564/10000
10/10 [==============================] - 

Epoch 9649/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1717
Epoch 9650/10000
10/10 [==============================] - 0s 12ms/step - loss: 0.1717
Epoch 9651/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1717
Epoch 9652/10000
10/10 [==============================] - 0s 9ms/step - loss: 0.1717
Epoch 9653/10000
10/10 [==============================] - 0s 12ms/step - loss: 0.1717
Epoch 9654/10000
10/10 [==============================] - 0s 11ms/step - loss: 0.1717
Epoch 9655/10000
10/10 [==============================] - 0s 12ms/step - loss: 0.1717
Epoch 9656/10000
10/10 [==============================] - 0s 11ms/step - loss: 0.1717
Epoch 9657/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1717
Epoch 9658/10000
10/10 [==============================] - 0s 11ms/step - loss: 0.1717
Epoch 9659/10000
10/10 [==============================] - 0s 11ms/step - loss: 0.1717
Epoch 9660/10000
10/10 [==============================]

Epoch 9745/10000
10/10 [==============================] - 0s 11ms/step - loss: 0.1715
Epoch 9746/10000
10/10 [==============================] - 0s 11ms/step - loss: 0.1715
Epoch 9747/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1716
Epoch 9748/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1716
Epoch 9749/10000
10/10 [==============================] - 0s 11ms/step - loss: 0.1716
Epoch 9750/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1717
Epoch 9751/10000
10/10 [==============================] - 0s 12ms/step - loss: 0.1718
Epoch 9752/10000
10/10 [==============================] - 0s 12ms/step - loss: 0.1720
Epoch 9753/10000
10/10 [==============================] - 0s 11ms/step - loss: 0.1723
Epoch 9754/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1729
Epoch 9755/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1737
Epoch 9756/10000
10/10 [==============================

Epoch 9841/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1716
Epoch 9842/10000
10/10 [==============================] - 0s 12ms/step - loss: 0.1715
Epoch 9843/10000
10/10 [==============================] - 0s 12ms/step - loss: 0.1715
Epoch 9844/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1716
Epoch 9845/10000
10/10 [==============================] - 0s 12ms/step - loss: 0.1716
Epoch 9846/10000
10/10 [==============================] - 0s 9ms/step - loss: 0.1715
Epoch 9847/10000
10/10 [==============================] - 0s 12ms/step - loss: 0.1715
Epoch 9848/10000
10/10 [==============================] - 0s 12ms/step - loss: 0.1715
Epoch 9849/10000
10/10 [==============================] - 0s 11ms/step - loss: 0.1716
Epoch 9850/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1715
Epoch 9851/10000
10/10 [==============================] - 0s 11ms/step - loss: 0.1715
Epoch 9852/10000
10/10 [==============================]

Epoch 9937/10000
10/10 [==============================] - 0s 12ms/step - loss: 0.1714
Epoch 9938/10000
10/10 [==============================] - 0s 14ms/step - loss: 0.1714
Epoch 9939/10000
10/10 [==============================] - 0s 13ms/step - loss: 0.1714
Epoch 9940/10000
10/10 [==============================] - 0s 14ms/step - loss: 0.1714
Epoch 9941/10000
10/10 [==============================] - 0s 12ms/step - loss: 0.1714
Epoch 9942/10000
10/10 [==============================] - 0s 12ms/step - loss: 0.1714
Epoch 9943/10000
10/10 [==============================] - 0s 13ms/step - loss: 0.1714
Epoch 9944/10000
10/10 [==============================] - 0s 11ms/step - loss: 0.1714
Epoch 9945/10000
10/10 [==============================] - 0s 12ms/step - loss: 0.1714
Epoch 9946/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1714
Epoch 9947/10000
10/10 [==============================] - 0s 10ms/step - loss: 0.1714
Epoch 9948/10000
10/10 [==============================

In [69]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_mapped_input = decoder_embedding(decoder_inputs)
decoder_outputs, state_h, state_c = decoder_lstm(decoder_mapped_input, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = Flatten()(decoder_outputs)
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
decoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 42)                0         
_________________________________________________________________
embedding_13 (Embedding)     (None, 42, 256)           66048     
_________________________________________________________________
lstm_13 (LSTM)               [(None, 256), (None, 256) 525312    
Total params: 591,360
Trainable params: 591,360
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_18 (InputLayer)           (None, 14)           0                                            
__________________________________________________________________________________________________
embe

In [60]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, max_decoder_seq_length), dtype='int32')
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = word_id_dict['\t']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    i = 1
    id_word_dict_cpy = id_word_dict.copy()
    id_word_dict_cpy[0] = '0'
    while not stop_condition:
#         print(target_seq)
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
#         print("output_token:", output_tokens)
#         sampled_char = id_word_dict[int(output_tokens)]
        decoded_sentence = str([id_word_dict_cpy[int(round(output_token))] for output_token in output_tokens[0]])
        if i > 14:
            stop_condition = True
        else:
            target_seq[0, 1:i] = output_tokens[0, 1:i]
            i += 1
#         # Exit condition: either hit max length
#         # or find stop character.
#         if (sampled_char == '\n' or
#            len(decoded_sentence) > max_decoder_seq_length):
#             stop_condition = True

#         # Update the target sequence (of length 1).
#         target_seq[0, i] = output_tokens
#         i += 1

#         # Update states
#         states_value = [h, c]

    return decoded_sentence

In [61]:
for seq_index in range(num_samples):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: indonesia’s finance minister said on monday the country will seek to host the headquarters for the asian infrastructure investment bank (aiib), a new multi-lateral lender that is led by china with beijing wanting the role for itself. 

Decoded sentence: ['indonesia', 'rise', 'the', 'has', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
-
Input sentence: the san jose earthquakes became the latest team in major league soccer to move into their own purpose-built venue and they celebrated the opening game at the avaya stadium on sunday with a 2-1 win over the chicago fire. 

Decoded sentence: ['earthquakes', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
-
Input sentence: world number one novak djokovic fought off a determined challenge from swiss great roger federer on sunday to claim back-to-back titles at indian wells and celebrate his 50th tournament win on the atp tour. 

Decoded sentence: ['djokovic', '0', '0', '0', '0', '0', '0', '0', '0', '0', 